In [97]:
import pandas as pd
import datetime
import numpy as np

In [123]:
wines = pd.read_excel("data/Subset1.xls", 1)
print(min(wines['DOY']))

213.0


On ne peut donc au maximum prendre comme data que les 213 jours précédent la récolte, une information supplémentaire n'est probablement pas importante car on va surtout analyser l'effet de la météo d'une année sur le vin

In [99]:
wines.dropna(subset = ['Harvest date'], inplace=True)
wines.reset_index(drop=True, inplace=True)

In [100]:
weather = pd.read_excel("data/Subset3.xls", [1,2,3,4,5])
weather[1]

In [102]:
i = list(range(1,201))
i = list(map(str, i))
temperatures = pd.DataFrame(columns = i)

In [103]:
for i in range(len(wines)):
    new_row = []
    r = wines["Reg"][i]
    d = wines['Harvest date'][i]
    h = int(wines['DOY'][i])
    new_row = list(weather[r].loc[(weather[r]['Year'] == d.year) & (weather[r]['DOY'].isin(range(h-200,h)))]['Tave'])
    if len(new_row) != 200:
        new_row = [float('nan')] * 200
    temperatures.loc[len(temperatures)] = new_row
print(temperatures.isnull().sum(axis = 0))

1      102
2      102
3      102
4      102
5      102
      ... 
196    102
197    102
198    102
199    102
200    102
Length: 200, dtype: int64


In [105]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(temperatures.values.ravel().reshape(-1, 1))

for col in temperatures.columns:
    temperatures[col] = scaler.transform(temperatures[[col]])

temperatures

/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/augustin/Documents/M1/projectML/project/projet_wine/.conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: 

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,-1.068493,-0.671793,-1.170211,-1.851721,-2.146703,-1.912752,-2.075500,-2.085672,-1.353303,-1.007462,...,0.009717,0.406417,0.630197,0.792945,0.070748,-0.112344,-0.000454,-0.041142,0.579338,1.077756
1,-1.516052,-1.577082,-1.414334,-1.627941,-1.465193,-1.861893,-1.800862,-1.404162,-1.414334,-1.058321,...,-0.275093,-0.448013,0.182638,0.345386,0.691227,0.965866,0.792945,0.691227,0.396245,0.853976
2,-1.465193,-1.007462,-0.722652,-1.800862,-1.963611,-1.800862,-1.739831,-1.912752,-2.421341,-2.197562,...,0.345386,0.009717,-0.051313,-0.498872,0.060576,0.579338,0.569166,-0.112344,-0.386983,-0.509044
3,-1.800862,-1.861893,-1.404162,-2.136531,-2.024641,-2.248421,-1.963611,-2.024641,-1.292272,-0.956603,...,-0.112344,-0.275093,-0.051313,-0.336124,0.121607,0.406417,0.396245,-0.325952,-0.224234,0.569166
4,-0.783683,-1.180383,-1.292272,-0.661621,-1.068493,-1.068493,-1.516052,-2.075500,-2.024641,-1.739831,...,-0.051313,0.172466,0.233497,0.406417,0.294528,0.345386,0.569166,0.742086,0.630197,0.406417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,-1.404162,-0.956603,-1.516052,-1.343131,-1.180383,-0.895572,-0.325952,-0.325952,-0.448013,-1.404162,...,-0.559903,-0.722652,-0.102172,0.111435,0.457276,-0.112344,0.457276,0.284356,0.284356,-0.224234
1469,-0.661621,-0.783683,-0.722652,-0.722652,-0.722652,-1.180383,-0.834542,-1.017634,-1.292272,-1.058321,...,0.569166,-0.437842,-0.102172,0.853976,0.467448,0.009717,-0.895572,-1.068493,-0.498872,-0.386983
1470,-0.610762,-1.180383,-1.068493,-1.119352,-1.068493,-0.386983,-0.610762,-0.732824,-1.292272,-0.895572,...,-0.783683,-0.895572,-0.620934,-1.353303,-1.180383,-0.783683,-0.783683,-1.404162,-2.258593,-2.645121
1471,-1.800862,-1.800862,-1.231241,-1.414334,-0.946431,-0.895572,-1.241413,-1.526223,-1.007462,-1.292272,...,-0.549731,-1.068493,-1.017634,-0.498872,-0.163203,0.345386,0.182638,0.070748,0.182638,0.009717


In [106]:
dataset = pd.concat([wines,temperatures], axis = 1)

,Table.No.,RecReg,Reg,DOY,Must.Brix,Must.Tacid,Must.pH,Wine.Alcohol,Wine.Facid,Wine.Extract,...,191,192,193,194,195,196,197,198,199,200
count,1473.000000,448.000000,1473.000000,1473.000000,1471.000000,1461.000000,1243.000000,1458.000000,1455.000000,1450.000000,...,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000
mean,17.843177,2.669643,3.390360,266.731161,22.638137,0.622327,3.498327,12.861728,0.488351,3.871241,...,0.349001,0.379510,0.360507,0.345646,0.328896,0.268304,0.247238,0.212873,0.203531,0.195666
std,3.840464,1.255954,1.247428,18.883646,2.304543,0.182924,0.279841,2.979097,0.130772,3.283334,...,0.766919,0.784353,0.788188,0.816135,0.828307,0.829336,0.819188,0.824220,0.808586,0.830017
min,9.000000,1.000000,1.000000,213.000000,14.200000,0.190000,2.760000,7.600000,0.200000,1.500000,...,-2.197562,-1.800862,-2.136531,-2.024641,-2.258593,-2.655293,-2.421341,-2.360311,-2.258593,-2.655293
25%,15.000000,2.000000,2.000000,255.000000,21.200000,0.500000,3.290000,10.900000,0.390000,2.300000,...,-0.163203,-0.125059,-0.112344,-0.224234,-0.224234,-0.275093,-0.275093,-0.359010,-0.336124,-0.374268
50%,19.000000,2.000000,4.000000,268.000000,22.700000,0.590000,3.490000,12.100000,0.480000,2.700000,...,0.233497,0.294528,0.294528,0.284356,0.208067,0.233497,0.179577,0.172466,0.172466,0.172466
75%,21.000000,4.000000,4.000000,280.000000,24.100000,0.720000,3.690000,13.400000,0.570000,3.100000,...,0.742086,0.758084,0.691227,0.792945,0.742086,0.681056,0.681056,0.626147,0.579338,0.630197
max,25.000000,5.000000,5.000000,336.000000,30.300000,1.630000,4.400000,22.200000,1.010000,18.500000,...,2.949366,2.949366,2.552666,2.664555,2.888335,2.949366,2.888335,2.949366,2.949366,2.552666


In [108]:
# Use pandas get_dummies() function to one-hot encode the 'fruits' column
one_hot_data = pd.get_dummies(dataset['Cultivar'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
dataset2 = pd.concat([dataset, one_hot_data], axis=1)

# Drop the original 'fruits' column
dataset2.drop('Cultivar', axis=1, inplace=True)

# Print the resulting DataFrame with one-hot encoded column
dataset2

,Table.No.,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,...,Trousseau,Valdepes,Verccia Sarda,Verccia bianca,Verdelho,Vermentino Favorita,Walschriesling,White Riesling,Zinfandel,sa Veltliner
0,9,White,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,...,0,0,0,0,0,0,0,0,0,0
1,9,White,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,...,0,0,0,0,0,0,0,0,0,0
2,9,White,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,...,0,0,0,0,0,0,0,0,0,0
3,9,White,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,...,0,0,0,0,0,0,0,0,0,0
4,9,White,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,...,0,0,0,0,0,0,0,0,0,0
1469,25,Red,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,...,0,0,0,0,0,0,0,0,0,0
1470,25,Red,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,...,0,0,0,0,0,0,0,0,0,0
1471,25,Red,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,...,0,0,0,0,0,0,0,0,0,0


### dataset2 ne sera probablement pas utile. En effet on augmente beaucoup trop la dimension par rapport à la quantité de données.

problem : there are a lot of different categories so there are many dimensions added. We already had a dimension to high compared to the amount of observations so this pnly increases the problem.

In [109]:
# Use pandas get_dummies() function to one-hot encode the 'fruits' column
one_hot_data3 = pd.get_dummies(dataset['Color'])

# Concatenate the one-hot encoded DataFrame with the original DataFrame
dataset3 = pd.concat([dataset, one_hot_data3], axis=1)

# Drop the original 'fruits' column
dataset3.drop('Color', axis=1, inplace=True)

# Print the resulting DataFrame with one-hot encoded column
dataset3

,Table.No.,Cultivar,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,Must.pH,...,193,194,195,196,197,198,199,200,Red,White
0,9,Chardony,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,3.88,...,0.630197,0.792945,0.070748,-0.112344,-0.000454,-0.041142,0.579338,1.077756,0,1
1,9,Chardony,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,3.79,...,0.182638,0.345386,0.691227,0.965866,0.792945,0.691227,0.396245,0.853976,0,1
2,9,Chardony,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,3.38,...,-0.051313,-0.498872,0.060576,0.579338,0.569166,-0.112344,-0.386983,-0.509044,0,1
3,9,Chardony,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,3.69,...,-0.051313,-0.336124,0.121607,0.406417,0.396245,-0.325952,-0.224234,0.569166,0,1
4,9,Chardony,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,3.19,...,0.233497,0.406417,0.294528,0.345386,0.569166,0.742086,0.630197,0.406417,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,3.74,...,-0.102172,0.111435,0.457276,-0.112344,0.457276,0.284356,0.284356,-0.224234,1,0
1469,25,Red Malaga (Molinera),Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,3.90,...,-0.102172,0.853976,0.467448,0.009717,-0.895572,-1.068493,-0.498872,-0.386983,1,0
1470,25,Ribier,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,NaN,...,-0.620934,-1.353303,-1.180383,-0.783683,-0.783683,-1.404162,-2.258593,-2.645121,1,0
1471,25,Ribier,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,3.58,...,-1.017634,-0.498872,-0.163203,0.345386,0.182638,0.070748,0.182638,0.009717,1,0


### On a seulement deux catégories donc la notion d'équidistence entre chaque catégorie n'est pas forcément importante, on peut se contenter d'encoder par 0 1 ou -1 1 et par la même occasion ne pas augmenter non nécessairement la dimension.

In [110]:
mapping = {'White': -1, 'Red': 1}

# Apply the mapping to the 'category' column using the map() function
dataset['Color'] = dataset['Color'].map(mapping)

dataset

,Table.No.,Cultivar,Color,Recommend,RecReg,Reg,Harvest date,DOY,Must.Brix,Must.Tacid,...,191,192,193,194,195,196,197,198,199,200
0,9,Chardony,-1,Particularly recommended,1.0,3,1936-09-08,251.0,27.2,0.54,...,0.009717,0.406417,0.630197,0.792945,0.070748,-0.112344,-0.000454,-0.041142,0.579338,1.077756
1,9,Chardony,-1,Particularly recommended,1.0,4,1936-08-31,243.0,23.5,0.67,...,-0.275093,-0.448013,0.182638,0.345386,0.691227,0.965866,0.792945,0.691227,0.396245,0.853976
2,9,Chardony,-1,Particularly recommended,1.0,1,1937-10-01,273.0,22.1,0.72,...,0.345386,0.009717,-0.051313,-0.498872,0.060576,0.579338,0.569166,-0.112344,-0.386983,-0.509044
3,9,Chardony,-1,Particularly recommended,1.0,3,1937-09-11,253.0,24.6,0.63,...,-0.112344,-0.275093,-0.051313,-0.336124,0.121607,0.406417,0.396245,-0.325952,-0.224234,0.569166
4,9,Chardony,-1,Particularly recommended,1.0,1,1938-09-14,256.0,19.8,0.78,...,-0.051313,0.172466,0.233497,0.406417,0.294528,0.345386,0.569166,0.742086,0.630197,0.406417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,25,Red Malaga (Molinera),1,Table grape,NaN,4,1937-10-25,297.0,21.8,0.47,...,-0.559903,-0.722652,-0.102172,0.111435,0.457276,-0.112344,0.457276,0.284356,0.284356,-0.224234
1469,25,Red Malaga (Molinera),1,Table grape,NaN,4,1940-10-07,280.0,22.7,0.36,...,0.569166,-0.437842,-0.102172,0.853976,0.467448,0.009717,-0.895572,-1.068493,-0.498872,-0.386983
1470,25,Ribier,1,Table grape,NaN,4,1935-11-01,304.0,20.2,0.55,...,-0.783683,-0.895572,-0.620934,-1.353303,-1.180383,-0.783683,-0.783683,-1.404162,-2.258593,-2.645121
1471,25,Ribier,1,Table grape,NaN,4,1937-10-13,285.0,17.6,0.50,...,-0.549731,-1.068493,-1.017634,-0.498872,-0.163203,0.345386,0.182638,0.070748,0.182638,0.009717


In [111]:
from sklearn.feature_extraction import FeatureHasher


# Instantiate a FeatureHasher object with the desired number of output features (n_features)
hasher = FeatureHasher(n_features=20, input_type='string')
cultivar_list = [[x] for x in dataset['Cultivar'].tolist()]
# Perform feature hashing on the 'fruits' column
hashed_features = hasher.fit_transform(cultivar_list)

# Convert the hashed features to a dense NumPy array and create a DataFrame
hashed_features_df = pd.DataFrame(hashed_features.toarray())

# Concatenate the hashed features DataFrame with the original DataFrame
dataset4 = pd.concat([dataset, hashed_features_df], axis=1)

# Drop the original 'fruits' column
dataset4.drop('Cultivar', axis=1, inplace=True)


### On utilisera cet encodage par la suite dans certaines analyses. la méthode par arbre nous permettra quand à elle de rester sur des données encodées par des str

In [112]:
dataset.nunique(axis=0)

Table.No.     17
Cultivar     148
Color          2
Recommend      5
RecReg         5
            ... 
196          189
197          185
198          185
199          185
200          190
Length: 216, dtype: int64

In [113]:
print(dataset.isna().sum()) #il faudrait trouver et enlever les 102 wines ou on a pas la temperature selon mon avis 

Table.No.       0
Cultivar        0
Color           0
Recommend       0
RecReg       1025
             ... 
196           102
197           102
198           102
199           102
200           102
Length: 216, dtype: int64


# Mise en place d'une analyse avec tout d'abord la color du wine comme attribute et le ph à prédire

In [114]:
copie = dataset.copy()
#enlever les colonnes qu'on ne compte pas utiliser
copie = copie.drop(columns = ['Table.No.', 'Cultivar', 'Recommend', 'RecReg', 'Reg', 'Wine.Alcohol',
       'Harvest date', 'DOY', 'Must.Brix', 'Must.Tacid', 'Wine.Facid', 'Wine.Extract', 'Wine.Tannin',
       'Wine.Tasting'])
#verifier qu'on a nos colonnes désirées
copie.columns 

Index(['Color', 'Must.pH', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '191', '192', '193', '194', '195', '196', '197', '198', '199', '200'],
      dtype='object', length=202)

In [115]:
columns = ['Color', 'Must.pH']
for i in range(1, 201):
    columns.append(f"{i}") 
#delete row with value = Nan
#voir si pour les y on peut pas remplacer par une valeur :
# d'autres possibilités (ffill et bfill qui propagent les valeurs observées aux voisines Nan)
# imputation par la moyenne (possible aussi avec la mediane)
# exemple :           movies_df['budget'].fillna(movies_df['budget'].mean(), inplace=True)
copie.dropna(subset = columns, inplace=True)
copie.reset_index(drop=True, inplace=True)

In [116]:
# selection of class to predict : here ph
y = copie[['Must.pH']].values


# temperature and color extraction
colonnes = ['Color']  # initialisation de la liste des noms de colonnes
for i in range(1, 201):
    colonnes.append(f"{i}")  

# récupération des valeurs de toutes les colonnes à la suite
X = copie[colonnes].values

### a definir si on veut faire avec adaline perceptron svm ....
### code exemple mais pas utilisé pour l'instant, on fera peutêtre un cross valisation entre les lasso, cv , ... 

```python
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, accuracy_score

# Définition des paramètres de la K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)  # objet KFold

# Initialisation de la liste des erreurs pour chaque fold
fold_errors = []

# Boucle sur chaque fold de la K-fold cross-validation
for train_index, test_index in kf.split(X):
    # Sélection des données d'apprentissage et de test pour ce fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Entraînement du modèle sur les données d'apprentissage
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Évaluation du modèle sur les données de test
    y_pred = model.predict(X_test)
    fold_error = mean_squared_error(y_test, y_pred)
    fold_errors.append(fold_error)

# Calcul de la moyenne et de l'écart-type des erreurs pour tous les folds
mean_error = sum(fold_errors) / len(fold_errors)
std_error = np.std(fold_errors)

# Affichage des résultats
print(f"K-fold cross-validation ({10} folds) :")
print(f"  Mean squared error : {mean_error:.2f}")
print(f"  Standard deviation : {std_error:.2f}")
```

# Analyse sur la donnée de tous les jours avec regularization. 
## on fait 
* Linéaire (pas vraiment)
* Ridge Regression
* Lasso Regression
## reste à faire 
* polynomial
* kernelized regression
* Random Forest Regression/ Decision Tree Regression
* Support Vector Regression
* (Neural Network Regression)

* Analyser avec plus de données comme Tmin, Tmax, un mix

## À réctifier: 
* standardization sur seulement l'ensemble train,
* Implémenter les métriques (accuracy, recall, etc)

In [122]:
import numpy as np
from sklearn.linear_model import LassoCV, RidgeCV

alphas = np.logspace(-5, 2, num=100)
lasso_cv = LassoCV(alphas=alphas, cv=5)
ridge_cv = RidgeCV(alphas=alphas, cv=5)
lasso_cv.fit(X, y.ravel())
ridge_cv.fit(X, y.ravel())
print("Lasso Best alpha:", lasso_cv.alpha_)
print("Ridge Best alpha:", ridge_cv.alpha_)



Lasso Best alpha: 0.0018307382802953678
Ridge Best alpha: 100.0
